In [1]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

data = pd.read_csv('/kaggle/input/mlpr-split-encoded-data/holdout (1).csv')

# 1) Storm Prediction

In [2]:
# Define features
storm_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'has_tornado', 'has_hail', 'has_flood', 'has_wind',
    'has_tree', 'has_broken', 'has_blown', 'tmin', 'tmax', 'tavg', 'ppt',
    'MAGNITUDE_IMPUTED', 'STATE_FIPS'
]
target_col = 'is_storm_lagged'

# Load storm model and scaler
storm_model = joblib.load('/kaggle/input/storm_xgb_model/scikitlearn/default/1/storm_xgb_model (1).pkl')
scaler = joblib.load('/kaggle/input/storm_xgb_model/scikitlearn/default/1/storm_scaler (2).pkl')

# Prepare features and scale
X = data[storm_features].values  # Convert to NumPy array to avoid feature name warning
X_scaled = scaler.transform(X)    # Apply scaling

# Storm prediction
storm_preds = storm_model.predict(X_scaled)

# Add predictions to DataFrame
data['predicted_storm'] = storm_preds

# Evaluate model
y_true = data[target_col]
y_pred = data['predicted_storm']
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1_storm = f1_score(y_true, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Storm Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_storm:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

# Filter storm cases
storm_data = data[data['predicted_storm'] == 1].copy()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Storm Prediction Metrics:
Accuracy : 0.9294
Precision: 0.9345
Recall   : 0.9272
F1 Score : 0.9308

Confusion Matrix:
[[19091  1402]
 [ 1569 19993]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.93     20493
           1       0.93      0.93      0.93     21562

    accuracy                           0.93     42055
   macro avg       0.93      0.93      0.93     42055
weighted avg       0.93      0.93      0.93     42055



# 2) Severity Prediction

In [5]:
# ----------------- SEVERITY PREDICTION -----------------

# Define severity features
severity_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'has_tornado', 'has_hail', 'has_flood', 'has_wind',
    'has_tree', 'has_broken', 'has_blown', 'tmin', 'tmax', 'tavg',
    'EVENT_TYPE_encoded', 'stability_encoded', 'CZ_FIPS', 'WFO_encoded'
]
target_col = 'severity_class'

# Load severity model and scaler
severity_model = joblib.load('/kaggle/input/severity_lgb_model-1/scikitlearn/default/1/severity_lgb_model (1).pkl')
severity_scaler = joblib.load('/kaggle/input/severity_lgb_model-1/scikitlearn/default/1/severity_scaler (3).pkl')

# Prepare features and scale
X = storm_data[severity_features].values
X_scaled = severity_scaler.transform(X)

# Severity prediction
severity_preds = severity_model.predict(X_scaled)
storm_data['predicted_severity'] = severity_preds

# Evaluate severity model
y_true = storm_data[target_col]
y_pred = storm_data['predicted_severity']
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_severity = f1_score(y_true, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

severity_counts = y_true.value_counts().sort_index()  # Counts of true Low (0), Medium (1), High (2)
support_low = severity_counts.get(0, 0)
support_medium = severity_counts.get(1, 0)
support_high = severity_counts.get(2, 0)
total_support = support_low + support_medium + support_high

print("Severity Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_severity:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

# Filter severity cases
low_severity_data = storm_data[storm_data['predicted_severity'] == 0].copy()
med_severity_data = storm_data[storm_data['predicted_severity'] == 1].copy()
high_severity_data = storm_data[storm_data['predicted_severity'] == 2].copy()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Severity Prediction Metrics:
Accuracy : 0.9328
Precision: 0.9330
Recall   : 0.9328
F1 Score : 0.9329

Confusion Matrix:
[[6437  151   86    0]
 [ 127 5667  526    0]
 [  63  478 6073    0]
 [   5    0    1 1781]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      6674
           1       0.90      0.90      0.90      6320
           2       0.91      0.92      0.91      6614
          10       1.00      1.00      1.00      1787

    accuracy                           0.93     21395
   macro avg       0.94      0.94      0.94     21395
weighted avg       0.93      0.93      0.93     21395



# 3) Outage Prediction

In [6]:
import numpy as np

# Derive is_outage target (assuming customers_out > 0 indicates an outage)
for df in [low_severity_data, med_severity_data, high_severity_data]:
    df['is_outage'] = (df['customers_out'] > 0).astype(int)

# Extract non-storm rows from storm_df_with_predictions
non_storm_data = data[data['predicted_storm'] == 0].copy()
non_storm_data['is_outage'] = 0  # No outage for non-storm events
non_storm_data['predicted_severity'] = 10  # Placeholder for non-storm rows

# Randomly split non-storm rows across the three DataFrames
non_storm_split = np.array_split(non_storm_data.sample(frac=1, random_state=42), 3)
non_storm_low, non_storm_medium, non_storm_high = non_storm_split

# Augment DataFrames with non-storm rows
low_severity_data = pd.concat([low_severity_data, non_storm_low], ignore_index=True)
med_severity_data = pd.concat([med_severity_data, non_storm_medium], ignore_index=True)
high_severity_data = pd.concat([high_severity_data, non_storm_high], ignore_index=True)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
outage_features = [
    'tmin', 'tmax', 'tavg', 'stability_encoded',
    'EVENT_TYPE_encoded', 'duration_hours', 'desc_word_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_broken', 'has_blown', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
    'INJURIES_DIRECT', 'DEATHS_DIRECT', 'CZ_FIPS'
]
target_col = 'is_outage'

### a. Low

In [8]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/low_severity_lgm_model-1/scikitlearn/default/1/low_severity_lgm_model (1).pkl')
scaler = joblib.load('/kaggle/input/low_severity_lgm_model-1/scikitlearn/default/1/low_severity_scaler (3).pkl')

# ----------------- PREPARE FEATURES -----------------
X = low_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = low_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_low = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_low:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics:
Accuracy : 0.9879
Precision: 0.9799
Recall   : 0.9958
F1 Score : 0.9878

Confusion Matrix:
[[6781  135]
 [  28 6575]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      6916
           1       0.98      1.00      0.99      6603

    accuracy                           0.99     13519
   macro avg       0.99      0.99      0.99     13519
weighted avg       0.99      0.99      0.99     13519



### b. Medium

In [9]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/medium_severity_xgb_model/scikitlearn/default/1/medium_severity_xgb_model.pkl')
scaler = joblib.load('/kaggle/input/medium_severity_xgb_model/scikitlearn/default/1/medium_severity_scaler (1).pkl')

# ----------------- PREPARE FEATURES -----------------
X = med_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = med_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_medium = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_medium:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics:
Accuracy : 0.9747
Precision: 0.9587
Recall   : 0.9897
F1 Score : 0.9739

Confusion Matrix:
[[6630  268]
 [  65 6220]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      6898
           1       0.96      0.99      0.97      6285

    accuracy                           0.97     13183
   macro avg       0.97      0.98      0.97     13183
weighted avg       0.98      0.97      0.97     13183



### c. High

In [10]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/high_severity_rf_model-2/scikitlearn/default/1/high_severity_rf_model (2).pkl')
scaler = joblib.load('/kaggle/input/high_severity_rf_model-2/scikitlearn/default/1/high_severity_scaler (2).pkl')

# ----------------- PREPARE FEATURES -----------------
X = high_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = high_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_high = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics (High Severity):")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_high:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics (High Severity):
Accuracy : 0.9757
Precision: 0.9640
Recall   : 0.9874
F1 Score : 0.9755

Confusion Matrix:
[[6664  246]
 [  84 6578]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      6910
           1       0.96      0.99      0.98      6662

    accuracy                           0.98     13572
   macro avg       0.98      0.98      0.98     13572
weighted avg       0.98      0.98      0.98     13572



In [11]:
# Compute weights based on true severity supports
w_low = support_low / total_support
w_medium = support_medium / total_support
w_high = support_high / total_support

# Compute the overall evaluation metric
overall_metric = f1_storm * f1_severity * (w_low * f1_low + w_medium * f1_medium + w_high * f1_high)

# Print the results
print("\nOverall Model Evaluation:")
print(f"F1 Storm: {f1_storm:.4f}")
print(f"F1 Severity (macro): {f1_severity:.4f}")
print(f"Weights - Low: {w_low:.3f}, Medium: {w_medium:.3f}, High: {w_high:.3f}")
print(f"F1 Low: {f1_low:.4f}, F1 Medium: {f1_medium:.4f}, F1 High: {f1_high:.4f}")
print(f"Overall Metric (F1_storm * F1_severity * (wL*F1L + wM*F1M + wH*F1H)): {overall_metric:.4f}")


Overall Model Evaluation:
F1 Storm: 0.9308
F1 Severity (macro): 0.9329
Weights - Low: 0.340, Medium: 0.322, High: 0.337
F1 Low: 0.9878, F1 Medium: 0.9739, F1 High: 0.9755
Overall Metric (F1_storm * F1_severity * (wL*F1L + wM*F1M + wH*F1H)): 0.8503
